In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import fastai
from fastai import *
from fastai.vision import *
from upassai import *
fastai.__version__

'1.0.28'

## Prepare data

In [6]:
vggface2_light_url = '../data/vggface2/vggface2-light'

In [17]:
faces_valid_df = compile_faces_dataset(vggface2_light_url, qty=300)
faces_valid_df.head()

,source,target,similarity
0,valid/n000178/0186_02.jpg,valid/n000178/0008_01.jpg,0
1,valid/n000178/0186_02.jpg,valid/n003540/0223_01.jpg,1
2,valid/n000178/0004_01.jpg,valid/n000178/0108_01.jpg,0
3,valid/n000178/0004_01.jpg,valid/n001687/0169_01.jpg,1
4,valid/n000178/0067_01.jpg,valid/n000178/0075_01.jpg,0


In [18]:
faces_train_df = compile_faces_dataset(
    vggface2_light_url, folder='train', qty=500)
faces_train_df.head()

,source,target,similarity
0,train/n004148/0079_01.jpg,train/n004148/0008_01.jpg,0
1,train/n004148/0079_01.jpg,train/n002492/0240_01.jpg,1
2,train/n004148/0004_01.jpg,train/n004148/0108_01.jpg,0
3,train/n004148/0004_01.jpg,train/n006032/0016_01.jpg,1
4,train/n004148/0104_01.jpg,train/n004148/0075_01.jpg,0


In [19]:
faces_df = pd.concat([faces_valid_df, faces_train_df])
faces_df.describe()

,source,target,similarity
count,800,800,800
unique,400,717,2
top,valid/n002245/0067_01.jpg,train/n009248/0016_01.jpg,1
freq,2,4,400


In [20]:
source_bunch = create_bunch(faces_df, cols='source', path=vggface2_light_url)
target_bunch = create_bunch(faces_df, cols='target', path=vggface2_light_url)

In [21]:
train_ds = TwinDataset(source_bunch.train_ds, target_bunch.train_ds)
valid_ds = TwinDataset(source_bunch.valid_ds, target_bunch.valid_ds)

In [22]:
twin_databunch = DataBunch.create(
    train_ds=train_ds, valid_ds=valid_ds, path=vggface2_light_url, bs=2)

## Train

In [23]:
learner = Learner(twin_databunch, SiameseNetwork(), loss_func=contrastive_loss)
print(learner)

Learner(data=DataBunch;
Train: <upassai.data.TwinDataset object at 0x12d677d30>;
Valid: <upassai.data.TwinDataset object at 0x12d677ba8>;
Test: None, model=SiameseNetwork(
  (cnn1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): Batch

In [24]:
learner.fit_one_cycle(2)

Total time: 55:00
epoch  train_loss  valid_loss
1      1.695551    1.580689    (28:56)
2      1.515909    1.380568    (26:03)

train_end
